In [1]:
%load_ext autoreload
%autoreload 2

import os
ROOT_DIR = os.getcwd()[:os.getcwd().rfind('NVcenter')]+ 'NVcenter'
os.chdir(ROOT_DIR)

from NVcenter import *
plt.style.use('NVcenter-default')

import qutip as q
import numpy as np 
import scipy.linalg as la

In [20]:
import qutip as q
import numpy as np

T2 = 5e-6
T1 = 100e-6

xm, xp = q.sigmax().eigenstates()[1]
xp, xm = xp * xp.dag(), xm * xm.dag()
ym, yp = q.sigmay().eigenstates()[1]
yp, ym = yp * yp.dag(), ym * ym.dag()
zp, zm = q.sigmaz().eigenstates()[1]
zm, zp = zp * zp.dag(), zm * zm.dag()

hada = 1/np.sqrt(2) * q.Qobj([[1,1], [1,-1]])
init_states = [hada * dm * hada for dm in [xp, xm, yp, ym, zp, zm]]

H = q.Qobj([[0,0], [0,0]])
c_ops = [np.sqrt(1/(2*T2)) * q.sigmaz()] # , np.sqrt(1/T1) * q.sigmap()]
times = np.linspace(0, 50e-6, 100)

states = []
for init_state in init_states:
    rho0 = init_state
    result = q.mesolve(H, rho0, times, c_ops).states
    result = [hada * dm * hada for dm in result]
    states.append(result)

pauli = True
if pauli:
    sigmax = np.array( [[q.expect(dm, q.sigmax()).real for dm in row] for row in states])
    sigmay = np.array( [[q.expect(dm, q.sigmay()).real for dm in row] for row in states])
    sigmaz = np.array( [[q.expect(dm, q.sigmaz()).real for dm in row] for row in states])
    miri_list = np.array([sigmax, sigmay, sigmaz])

else:
    pop0 = np.array( [[q.expect(dm, q.fock_dm(2, 0)).real for dm in row] for row in states] )
    pop1 = np.array( [[q.expect(dm, q.fock_dm(2, 1)).real for dm in row] for row in states] )
    miri_list = np.array([pop0, pop1])
    
miri_list = np.transpose(miri_list, (1, 0, 2))
np.save("Lindblad1.npy", miri_list)

In [21]:
PAULIS = [q.qeye(2), q.sigmax(), q.sigmay(), q.sigmaz()]
MATRIX_UNITS = [np.array([[1,0], [0,0]]), 
               np.array([[0,0], [1,0]]),
               np.array([[0,1], [0,0]]),
               np.array([[0,0], [0,1]])
               ]

def construct_PTM(input_array):
    """ input_array 
        1st dimension: pauli eigenstates as initial states in order xp,xm,yp,ym,zp,zm.
        2nd dimension: pauli observables in order X, Y, Z.
        3rd dimension: timesteps. 
    """

    n_timesteps = input_array.shape[-1]
    PTM = np.zeros((4,4,n_timesteps))

    PTM[0,0,:] = 1 # first column is (1,0,0,0)
    for i in range(3):
        PTM[i+1,0,:] = 0.5 * (input_array[4,i,:] + input_array[5,i,:]) # identity
        PTM[i+1,1,:] = 0.5 * (input_array[0,i,:] - input_array[1,i,:]) # sigma_x
        PTM[i+1,2,:] = 0.5 * (input_array[2,i,:] - input_array[3,i,:]) # sigma_y
        PTM[i+1,3,:] = 0.5 * (input_array[4,i,:] - input_array[5,i,:]) # sigma_z
    return PTM 

def apply_ptm(PTM, rho):
    pauli_vec_before = rho_to_pauli_vec(rho)
    pauli_vec_after = PTM @ pauli_vec_before
    rho = pauli_vec_to_rho(pauli_vec_after)
    return rho

def rho_to_pauli_vec(rho):
    return 0.5 * np.array([np.trace(rho @ P.full()) for P in PAULIS]).T

def pauli_vec_to_rho(pauli_vec):
    return sum([c * P.full() for c, P in zip(pauli_vec, PAULIS)])

In [22]:
miri_list = np.load("Lindblad1.npy")
miri_list = miri_list[:,:,1:]
PTM_list = construct_PTM(miri_list)

init_state = q.fock_dm(2,0).full()
rho_list = []
for i in range(99):
    rho = apply_ptm(PTM_list[:,:,i], init_state)
    rho_list.append( rho ) 

A = create_mitigated_matrix_qutip(rho_list, miri_list, 99)

In [19]:
A[82]

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[1.00000000e+00-1.13522770e-17j 2.23026275e-09+2.81921826e-09j]
 [2.23026270e-09-2.81921823e-09j 4.60975148e-16-1.27216790e-17j]]

In [23]:
A[82]

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 7.90600855e-01-5.55137858e-15j  1.14182005e-14+2.56095148e-01j]
 [-3.72939846e-15-2.56095148e-01j  2.09399145e-01+2.07287807e-14j]]